In [118]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchtnt.utils.tqdm import create_progress_bar
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [80]:
torch.manual_seed(67)

In [81]:
check_gpu = torch.cuda.is_available()
device = torch.device("cpu")

if check_gpu:
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    
print(f'Using device: {device}')

Using device: cuda


In [82]:
df = pd.read_csv('../data/wave_packet_spread.csv')

In [83]:
df.head()

,h_bar,mass,time,sig_0,sig_0_2,sig_t
0,1,1,4.756226,9.381908,88.020205,9.385332
1,1,1,0.680782,0.188217,0.035426,1.818267
2,1,1,5.368285,5.265139,27.721690,5.289762
3,1,1,1.418382,3.975479,15.804436,3.979480
4,1,1,7.132206,9.257078,85.693495,9.265090


In [84]:
df.shape

(50000, 6)

##### <b>Wavepacket Spreading</b>

$$
\sigma(t) = \sigma_0 \, \sqrt{1 + \left(\frac{\hbar t}{2 m \sigma_0^2}\right)^2}
$$

The characteristic spreading time is defined as:

$$
t_c = \frac{2 m \sigma_0^2}{\hbar}
$$


##### Time-Dependent Wavepacket Width


Using the normalized time $\tfrac{t}{t_c}$, the wavepacket width becomes:

$$
\sigma(t) = \sigma_0 \, \sqrt{1 + \left(\frac{t}{t_c}\right)^2 }
$$

In [85]:
df['t_c'] =  (2 * df['mass'] * df['sig_0_2']) / df['h_bar'] # i know h bar's one gng 😭😂✌️

In [86]:
df['norm_time'] = df['time'] / df['t_c']

In [87]:
df['spreading_factor'] = 1 + np.square(df['norm_time'])

In [88]:
df

,h_bar,mass,time,sig_0,sig_0_2,sig_t,t_c,norm_time,spreading_factor
0,1,1,4.756226,9.381908,88.020205,9.385332,176.040411,0.027018,1.000730
1,1,1,0.680782,0.188217,0.035426,1.818267,0.070852,9.608565,93.324518
2,1,1,5.368285,5.265139,27.721690,5.289762,55.443380,0.096825,1.009375
3,1,1,1.418382,3.975479,15.804436,3.979480,31.608872,0.044873,1.002014
4,1,1,7.132206,9.257078,85.693495,9.265090,171.386989,0.041615,1.001732
...,...,...,...,...,...,...,...,...,...
49995,1,1,3.329266,1.077968,1.162014,1.883260,2.324029,1.432541,3.052172
49996,1,1,1.558871,5.098636,25.996091,5.100927,51.992182,0.029983,1.000899
49997,1,1,5.244002,5.927721,35.137876,5.944201,70.275751,0.074620,1.005568
49998,1,1,6.712440,3.603203,12.983073,3.721650,25.966145,0.258507,1.066826


In [89]:
features = df.columns.tolist()
features

['h_bar',
 'mass',
 'time',
 'sig_0',
 'sig_0_2',
 'sig_t',
 't_c',
 'norm_time',
 'spreading_factor']

In [90]:
y_feat = features.pop(features.index('sig_t'))
x_feat = features

In [91]:
print(f'X_features: {x_feat}, y_features: { y_feat}')

X_features: ['h_bar', 'mass', 'time', 'sig_0', 'sig_0_2', 't_c', 'norm_time', 'spreading_factor'], y_features: sig_t


In [92]:
X = df[x_feat].values
y = df[y_feat].values

In [93]:
X

array([[1.00000000e+00, 1.00000000e+00, 4.75622563e+00, ...,
        1.76040411e+02, 2.70178058e-02, 1.00072996e+00],
       [1.00000000e+00, 1.00000000e+00, 6.80781711e-01, ...,
        7.08515501e-02, 9.60856482e+00, 9.33245179e+01],
       [1.00000000e+00, 1.00000000e+00, 5.36828490e+00, ...,
        5.54433802e+01, 9.68246322e-02, 1.00937501e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 5.24400229e+00, ...,
        7.02757515e+01, 7.46203658e-02, 1.00556820e+00],
       [1.00000000e+00, 1.00000000e+00, 6.71243982e+00, ...,
        2.59661454e+01, 2.58507365e-01, 1.06682606e+00],
       [1.00000000e+00, 1.00000000e+00, 7.90266995e+00, ...,
        1.98226178e+02, 3.98669340e-02, 1.00158937e+00]])

In [94]:
y

array([9.38533201, 1.81826659, 5.28976192, ..., 5.94420141, 3.72165021,
       9.96346408])

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=67)

In [96]:
X_train

array([[1.00000000e+00, 1.00000000e+00, 2.09499558e+00, ...,
        8.26443305e+01, 2.53495377e-02, 1.00064260e+00],
       [1.00000000e+00, 1.00000000e+00, 6.41435828e+00, ...,
        3.07784969e+00, 2.08403884e+00, 5.34321787e+00],
       [1.00000000e+00, 1.00000000e+00, 9.36743382e+00, ...,
        2.80982202e-01, 3.33381750e+01, 1.11243391e+03],
       ...,
       [1.00000000e+00, 1.00000000e+00, 8.39385446e+00, ...,
        4.46452458e+01, 1.88012280e-01, 1.03534862e+00],
       [1.00000000e+00, 1.00000000e+00, 6.35880252e+00, ...,
        1.31900861e+02, 4.82089537e-02, 1.00232410e+00],
       [1.00000000e+00, 1.00000000e+00, 5.89285185e-01, ...,
        6.32138596e+01, 9.32208836e-03, 1.00008690e+00]])

In [97]:
y_train

array([ 6.43029695,  2.86754437, 12.50148252, ...,  4.80746261,
        8.13041858,  5.62224835])

In [98]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.25, random_state=67)

In [99]:
X_train.shape

(30000, 8)

In [100]:
X_dev.shape

(10000, 8)

In [101]:
X_scaler = StandardScaler()

X_train = X_scaler.fit_transform(X_train)
X_dev = X_scaler.transform(X_dev)
X_test = X_scaler.transform(X_test)

In [102]:
y_train = y_train.reshape(-1, 1)  
y_dev   = y_dev.reshape(-1, 1)    
y_test  = y_test.reshape(-1, 1)   

print(f'''
      
      y_train shape: {y_train.shape}
      y_dev shape: {y_dev.shape}
      y_test shspe: {y_test.shape}
      
      ''')



      y_train shape: (30000, 1)
      y_dev shape: (10000, 1)
      y_test shspe: (10000, 1)

      


In [103]:
y_scaler = StandardScaler()

y_train = y_scaler.fit_transform(y_train)
y_dev = y_scaler.transform(y_dev)
y_test = y_scaler.transform(y_test)

In [104]:
class QWaveSet(Dataset):
    
    def __init__(self, features, labels):
        
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [105]:
data_train = QWaveSet(X_train, y_train)
data_dev = QWaveSet(X_dev, y_dev)
data_test = QWaveSet(X_test, y_test)

In [106]:
train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
dev_loader = DataLoader(data_dev, batch_size=32, shuffle=True)
test_loader = DataLoader(data_test, batch_size=32, shuffle=False)

In [107]:
f'train loader size : {len(train_loader)}' , f'dev loader size: {len(dev_loader)}'

('train loader size : 938', 'dev loader size: 313')

In [108]:
class QWaveModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
            nn.Linear(64,1)
            
        )
        
    def forward(self, X):
        return self.model(X)

In [121]:
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 32
epochs = 250

In [110]:
model = QWaveModel(X_train.shape[1])

model.to(device)

loss_fn = nn.MSELoss()

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
from torchtnt.utils.tqdm import create_progress_bar

num_steps_per_epoch = len(train_loader)

for epoch in range(epochs):

    pbar = create_progress_bar(
        dataloader=train_loader,
        desc=f"Epoch [{epoch+1}/{epochs}]",
        num_epochs_completed=epoch,
        num_steps_completed=0,
        max_steps=None,
        max_steps_per_epoch=num_steps_per_epoch,
    )

    for batch_idx, (batch_features, y_true) in enumerate(train_loader):
        batch_features, y_true = batch_features.to(device), y_true.to(device)

        y_pred = model(batch_features)
        loss = loss_fn(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.update(1)
        pbar.set_postfix(loss=loss.item())

    pbar.close() 
